In [73]:
import pandas as pd
from jaal import Jaal
from jaal.datasets import load_got
import matplotlib.pyplot as plt

In [74]:
sz_file_path = "./dump.csv"
sz_file_csv = pd.read_csv(sz_file_path,usecols=['initiatingDeviceName','receivingDeviceName', 'connectionCount'],
                          header=0,
                       
                        )

In [75]:
df = sz_file_csv.rename(
    columns={
        "initiatingDeviceName": "from",
        "receivingDeviceName": "to",
        "connectionCount": "weight"
    }
)

In [76]:
df_grouped = df.groupby(['from','to'], as_index=False)['weight'].sum()

In [77]:
df = sz_file_csv.rename(
    columns={
        "initiatingDeviceName": "id",
        "receivingDeviceName": "to",
        "connectionCount": "weight",

    }
)
df_nodes = df.groupby(['id'], as_index=False)['weight'].sum()

In [79]:
#Jaal(df_grouped).plot(port=8071)
#Jaal(df_grouped,df_nodes).plot(vis_opts={'height': '600px', # change height
#                                     'interaction':{'hover': True}, # turn on-off the hover                                    
#                                    'physics':{'enabled': False}})                     # Estatico

edge_df = df_grouped[['initiatingDeviceName', 'receivingDeviceName', 
                      'protocol', 'connectionCount']]

edge_df.columns = ['from', 'to', 'protocol', 'connectionCount']  # Rename


jaal = Jaal(edge_df, node_df, 
            edge_features=['protocol', 'connectionCount'],  # For filtering
            node_features=['groupType', 'groupName'])

# Plotting with enhanced interactions
jaal.plot(vis_opts={
    'height': '600px',
    'interaction': {
        'hover': True, 
        'hoverConnectedEdges': True,    # Highlight connected edges on hover
        'selectable': True,
        'selectConnectedEdges': True,   # Select connected edges on node click
        'multiselect': True             # Allow multiple node/edge selection
    },
    'physics': {'enabled': False},       # Keep it static
    'nodes': { 
        'shape': 'dot',
        'size': 10 
    },
    'edges': {
        'arrows': { 'to': {'enabled': True} }  # Show directionality if applicable
    }
},
search_enabled=True,                       # Enable node search
filter_enabled=True)                       # Enable filtering by features

# Custom JavaScript Callback for Protocol Filtering (example)
jaal.add_js_callback('node_selected', """
    function(nodes) {
        var selectedProtocol = document.getElementById('protocol-filter').value;
        var edges = this.body.data.edges;
        edges.update(edges.map(edge => {
            edge.hidden = (selectedProtocol !== 'all' && edge.protocol !== selectedProtocol);
            return edge;
        }));
    }
""")

# Add Protocol Filter Dropdown to the Jaal UI (example)
jaal.add_html_element("""
    <select id="protocol-filter">
        <option value="all">All Protocols</option>
        <option value="TCP">TCP</option>
        <option value="UDP">UDP</option>
        <option value="ICMP">ICMP</option>
        // ... (add more options for your protocols)
    </select>
""")



NameError: name 'edge_df' is not defined